In [2]:
import sys
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from dotenv import load_dotenv
import dask
from dask.distributed import Client
import dask.dataframe as dd

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

sys.dont_write_bytecode = True
from src.utils.utils import *
from src.utils.constants import *
from src.visualization.visualize import *

/Users/francescoperagine/Documents/Repository/big_data_bitcoin_forecasting/venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
load_dotenv()
ip = os.getenv("DISTRIBUTED_MAIN_IP")
client = Client(ip, threads_per_worker=2)

/Users/francescoperagine/Documents/Repository/big_data_bitcoin_forecasting/venv/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50056 instead
  warnings.warn(


In [4]:
ground_truth = pd.read_parquet(GROUND_TRUTH_PATH)
gt_to_merge = ground_truth.copy()
ground_truth = ground_truth[['origin_time', 'label']]

In [6]:

imported_data = {}

for exchange in EXCHANGES:
    imported_data[(CANDLES, exchange)] = dd.read_parquet(os.path.join(INTERIM_DATA_PATH, f'{exchange}_{CANDLES}_pca_data.parquet'))
    imported_data[(ORDERBOOKS, exchange)] = dd.read_parquet(os.path.join(INTERIM_DATA_PATH, f'{exchange}_{ORDERBOOKS}_pca_data.parquet'))

FileNotFoundError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: /Users/francescoperagine/Documents/Repository/big_data_bitcoin_forecasting/data/interim/BINANCE_candles_pca_data.parquet

In [ ]:
for(exchange, data_type), df in imported_data.items():
    print(f'{exchange} {data_type}: {df.shape}')

In [ ]:
merged_df = {}
cols_to_drop = ['origin_time', 'label']

for (data_type, exchange), df in imported_data.items():     
    merged_df[(data_type, exchange)] = {}
    merged_df[(data_type, exchange)]['full'] = pd.merge(ground_truth[cols_to_drop], df, on='origin_time', how='inner')
    merged_df[(data_type, exchange)]['X'] = merged_df[(data_type, exchange)]['full'].drop(cols_to_drop, axis=1)
    merged_df[(data_type, exchange)]['y'] = merged_df[(data_type, exchange)]['full']['label']

In [ ]:
data = {}

for (data_type, exchange), df in merged_df.items():
    data[(data_type, exchange)] = {}
    data[(data_type, exchange)]['X_train'], data[(data_type, exchange)]['X_test'], data[(data_type, exchange)]['y_train'], data[(data_type, exchange)]['y_test'] = train_test_split(df['X'], df['y'], test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
best_params = {}
best_score = {}
evaluation = {}

param_distributions = {
    'criterion': [RANDOM_FOREST_CRITERION],
    'max_depth': range(1, 20),
    'min_samples_split': [10, 20, 50, 100],
    'min_samples_leaf': [10, 20, 30, 50],
    'max_features': [None, 'auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 50, 100],
    'min_impurity_decrease': [0.0, 0.01, 0.05, 0.1]
}

In [ ]:
@dask.delayed
def process_dataset(data_type, exchange, df):
    clf = RandomForestClassifier(random_state=42)

    # Initialize the RandomizedSearchCV object
    randomized_search = RandomizedSearchCV(estimator=clf, param_distributions=param_distributions, n_iter=50, cv=CV_FOLDS, scoring='accuracy', n_jobs=-1, random_state=42)

    # Fit the random search to the data
    randomized_search.fit(df['X_train'], df['y_train'])

    # Best parameters and score
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_
    print(f"Best parameters for {exchange} {data_type}: {best_params}")

    # Use the best estimator found by RandomizedSearchCV
    best_clf = randomized_search.best_estimator_

    # Predict on the test set
    df['y_pred'] = best_clf.predict(df['X_test'])

    evaluation = {}

    # Calculate accuracy
    evaluation['accuracy'] = accuracy_score(df['y_test'], df['y_pred'])

    # Convert classification report to DataFrame
    report = classification_report(df['y_test'], df['y_pred'], target_names=['positive', 'neutral', 'negative'], digits=2, output_dict=True)
    evaluation['classification_report'] = pd.DataFrame(report).transpose()

    # Convert confusion matrix to DataFrame
    evaluation['confusion_matrix'] = pd.DataFrame(
        confusion_matrix(df['y_test'], df['y_pred']),
        index=['true:positive', 'true:neutral', 'true:negative'],
        columns=['pred:positive', 'pred:neutral', 'pred:negative']
    )
    pd.DataFrame.to_pickle(evaluation, os.path.join(PROCESSED_DATA_PATH, f'{exchange}_{data_type}_evaluation.pkl'))

    # Display results
    print(f'{exchange} {data_type} Accuracy: {evaluation["accuracy"]:.2f}')
    print(f'{exchange} {data_type} Classification Report:')
    display(evaluation['classification_report'])
    print(f'{exchange} {data_type} Confusion Matrix:')
    display(evaluation['confusion_matrix'])    

    # Calculate cross-validated train and test scores for plotting learning curves
    train_scores = []
    test_scores = []
    depths = param_distributions['max_depth']

    # Reuse cross-validation results from RandomizedSearchCV
    cv_results = pd.DataFrame(randomized_search.cv_results_)
    
    for depth in depths:
        subset = cv_results[cv_results['param_max_depth'] == depth]
        train_scores.append(subset['mean_train_score'].values[0])
        test_scores.append(subset['mean_test_score'].values[0])

    plot_tree_learning_curves(exchange, data_type, depths, train_scores, test_scores, 'decision_tree')

    pd.DataFrame.to_pickle(best_params, os.path.join(PROCESSED_DATA_PATH, f'random_forest_best_params.pkl'))
    pd.DataFrame.to_pickle(best_score, os.path.join(PROCESSED_DATA_PATH, f'random_forest_best_score.pkl'))

In [ ]:
tasks = [process_dataset(data_type, exchange, df) for (data_type, exchange), df in data.items()]
dask.compute(*tasks)